# js2xml -  Scraping JavaScript code with XPath

## Some JavaScript code from a real webpage

In [1]:
# taken from http://shopmarkethq.com/collections/accessories-shades/products/last-chance-black
markethq = ur'''
<script type="text/javascript">
  $(function() { 
    $product = $('#product-' + 3727646661);
    new Shopify.OptionSelectors("product-select-3727646661", { product: {"id":3727646661,"title":"Last Chance - Black","handle":"last-chance-black","description":"\u003cspan\u003e- Acrylic lenses \u003c\/span\u003e\u003cbr\u003e\u003cspan\u003e- Lightweight and impact resistant lenses \u003c\/span\u003e\u003cbr\u003e\u003cspan\u003e- Large, oversized circular shape \u003c\/span\u003e","published_at":"2016-02-10T13:53:00+11:00","created_at":"2016-02-10T13:53:46+11:00","vendor":"MINKPINK SUNGLASSES","type":"ACCESSORIES","tags":[],"price":4900,"price_min":4900,"price_max":4900,"available":true,"price_varies":false,"compare_at_price":null,"compare_at_price_min":0,"compare_at_price_max":0,"compare_at_price_varies":false,"variants":[{"id":10967944453,"title":"BLACK \/ ONE SIZE","option1":"BLACK","option2":"ONE SIZE","option3":null,"sku":"MNP1608012.BLK","requires_shipping":true,"taxable":true,"featured_image":null,"available":true,"options":["BLACK","ONE SIZE"],"price":4900,"weight":0,"compare_at_price":null,"inventory_quantity":2,"inventory_management":"shopify","inventory_policy":"deny","barcode":null}],"images":["\/\/cdn.shopify.com\/s\/files\/1\/0053\/6742\/products\/10.jpg?v=1455075040","\/\/cdn.shopify.com\/s\/files\/1\/0053\/6742\/products\/9.jpg?v=1455075040"],"featured_image":"\/\/cdn.shopify.com\/s\/files\/1\/0053\/6742\/products\/10.jpg?v=1455075040","options":["COLOUR","SIZE"],"content":"\u003cspan\u003e- Acrylic lenses \u003c\/span\u003e\u003cbr\u003e\u003cspan\u003e- Lightweight and impact resistant lenses \u003c\/span\u003e\u003cbr\u003e\u003cspan\u003e- Large, oversized circular shape \u003c\/span\u003e"}, onVariantSelected:
    selectCallback, enableHistoryState: true });

    
    jQuery('.swatch :radio').change(function() {
      var optionIndex = jQuery(this).parents('.swatch').attr('data-option-index');
      var optionValue = jQuery(this).val();
      jQuery('#product-3727646661 .single-option-selector').eq(optionIndex).find('option').filter(function() { return jQuery(this).text() == optionValue }).parent().val(optionValue).trigger('change');	   
     }); 
     
      
        $('.single-option-selector').selecter();  
  
      
       
   });       
</script>
'''

In [2]:
from IPython.display import IFrame
IFrame('http://shopmarkethq.com/collections/accessories-shades/products/last-chance-black', width='100%', height=350)

You feed the HTML into a Parsel selector

In [5]:
import parsel 

In [6]:
selector = parsel.Selector(text=markethq, type="html")

# get the text node for that <script> element
jssnippet = selector.xpath('string(//script/text())').extract_first()

# printing just some characters to check
print(jssnippet[:200] + '...')


  $(function() { 
    $product = $('#product-' + 3727646661);
    new Shopify.OptionSelectors("product-select-3727646661", { product: {"id":3727646661,"title":"Last Chance - Black","handle":"last-cha...


You build a nice custom regular expression to get what's after "product:"

In [7]:
import re

regex = re.compile(
    r'''
    "product-select-\d+",\s+
    {
        \s+product:\s(?P<json>.+),\s+
        onVariantSelected:\s+selectCallback
    ''', re.M|re.X)
m = regex.search(jssnippet)

In [ ]:
m.group('json')

You decode it with the `json` package

In [8]:
import json

json.loads(m.group('json'))

{u'available': True,
 u'compare_at_price': None,
 u'compare_at_price_max': 0,
 u'compare_at_price_min': 0,
 u'compare_at_price_varies': False,
 u'content': u'<span>- Acrylic lenses </span><br><span>- Lightweight and impact resistant lenses </span><br><span>- Large, oversized circular shape </span>',
 u'created_at': u'2016-02-10T13:53:46+11:00',
 u'description': u'<span>- Acrylic lenses </span><br><span>- Lightweight and impact resistant lenses </span><br><span>- Large, oversized circular shape </span>',
 u'featured_image': u'//cdn.shopify.com/s/files/1/0053/6742/products/10.jpg?v=1455075040',
 u'handle': u'last-chance-black',
 u'id': 3727646661,
 u'images': [u'//cdn.shopify.com/s/files/1/0053/6742/products/10.jpg?v=1455075040',
  u'//cdn.shopify.com/s/files/1/0053/6742/products/9.jpg?v=1455075040'],
 u'options': [u'COLOUR', u'SIZE'],
 u'price': 4900,
 u'price_max': 4900,
 u'price_min': 4900,
 u'price_varies': False,
 u'published_at': u'2016-02-10T13:53:00+11:00',
 u'tags': [],
 u'title

> "There must be a better way!"

― Raymond Hettinger

# Introducing js2xml

In [9]:
import js2xml

In [10]:
jscode = r"""
function factorial(n) {
     if (n === 0) {
         return 1;
     }
     return n * factorial(n - 1);
}"""

In [12]:
js2xml.parse(jscode)

<Element program at 0x7f47195ca998>

In [13]:
type(js2xml.parse(jscode))  

lxml.etree._Element

In [14]:
# you get a <program> tree
print(
        js2xml.pretty_print(
            js2xml.parse(jscode)
        )
)

<program>
  <funcdecl name="factorial">
    <parameters>
      <identifier name="n"/>
    </parameters>
    <body>
      <if>
        <predicate>
          <binaryoperation operation="===">
            <left>
              <identifier name="n"/>
            </left>
            <right>
              <number value="0"/>
            </right>
          </binaryoperation>
        </predicate>
        <then>
          <block>
            <return>
              <number value="1"/>
            </return>
          </block>
        </then>
      </if>
      <return>
        <binaryoperation operation="*">
          <left>
            <identifier name="n"/>
          </left>
          <right>
            <functioncall>
              <function>
                <identifier name="factorial"/>
              </function>
              <arguments>
                <binaryoperation operation="-">
                  <left>
                    <identifier name="n"/>
                  </left>
                

## Another look at the JavaScript snippet we saw before

In [15]:
print(jssnippet)


  $(function() { 
    $product = $('#product-' + 3727646661);
    new Shopify.OptionSelectors("product-select-3727646661", { product: {"id":3727646661,"title":"Last Chance - Black","handle":"last-chance-black","description":"<span>- Acrylic lenses <\/span><br><span>- Lightweight and impact resistant lenses <\/span><br><span>- Large, oversized circular shape <\/span>","published_at":"2016-02-10T13:53:00+11:00","created_at":"2016-02-10T13:53:46+11:00","vendor":"MINKPINK SUNGLASSES","type":"ACCESSORIES","tags":[],"price":4900,"price_min":4900,"price_max":4900,"available":true,"price_varies":false,"compare_at_price":null,"compare_at_price_min":0,"compare_at_price_max":0,"compare_at_price_varies":false,"variants":[{"id":10967944453,"title":"BLACK \/ ONE SIZE","option1":"BLACK","option2":"ONE SIZE","option3":null,"sku":"MNP1608012.BLK","requires_shipping":true,"taxable":true,"featured_image":null,"available":true,"options":["BLACK","ONE SIZE"],"price":4900,"weight":0,"compare_at_price":null

In [16]:
import js2xml

jstree = js2xml.parse(jssnippet)

In [17]:
# the "magical" getall()
data = js2xml.jsonlike.getall(jstree)

In [18]:
data

[{'enableHistoryState': True,
  'onVariantSelected': 'selectCallback',
  'product': {'available': True,
   'compare_at_price': None,
   'compare_at_price_max': 0,
   'compare_at_price_min': 0,
   'compare_at_price_varies': False,
   'content': '<span>- Acrylic lenses </span><br><span>- Lightweight and impact resistant lenses </span><br><span>- Large, oversized circular shape </span>',
   'created_at': '2016-02-10T13:53:46+11:00',
   'description': '<span>- Acrylic lenses </span><br><span>- Lightweight and impact resistant lenses </span><br><span>- Large, oversized circular shape </span>',
   'featured_image': '//cdn.shopify.com/s/files/1/0053/6742/products/10.jpg?v=1455075040',
   'handle': 'last-chance-black',
   'id': 3727646661,
   'images': ['//cdn.shopify.com/s/files/1/0053/6742/products/10.jpg?v=1455075040',
    '//cdn.shopify.com/s/files/1/0053/6742/products/9.jpg?v=1455075040'],
   'options': ['COLOUR', 'SIZE'],
   'price': 4900,
   'price_max': 4900,
   'price_min': 4900,
   '

Did we get the same data?

In [19]:
data[0]['product'] == json.loads(m.group('json'))

True

Getting arguments for a contructor

In [20]:
# recall this line:
# new Shopify.OptionSelectors("product-select-3727646661", { product...
print(js2xml.pretty_print(jstree))

<program>
  <functioncall>
    <function>
      <identifier name="$"/>
    </function>
    <arguments>
      <funcexpr>
        <identifier/>
        <parameters/>
        <body>
          <assign operator="=">
            <left>
              <identifier name="$product"/>
            </left>
            <right>
              <functioncall>
                <function>
                  <identifier name="$"/>
                </function>
                <arguments>
                  <binaryoperation operation="+">
                    <left>
                      <string>#product-</string>
                    </left>
                    <right>
                      <number value="3727646661"/>
                    </right>
                  </binaryoperation>
                </arguments>
              </functioncall>
            </right>
          </assign>
          <new>
            <dotaccessor>
              <object>
                <identifier name="Shopify"/>
              </object>


In [21]:
jstree.xpath('//new/arguments/*')

[<Element string at 0x7f47195caa70>, <Element object at 0x7f47195ca4d0>]

In [23]:
# just the 2nd argument of the contructor identified by "OptionSelectors"
jstree.xpath('//new[.//@name="OptionSelectors"]/arguments/*[2]')

In [25]:
# Note: make dict is not the most appropriate name here...
map(js2xml.jsonlike.make_dict, jstree.xpath('//new/arguments/*'))

['product-select-3727646661',
 {'enableHistoryState': True,
  'onVariantSelected': 'selectCallback',
  'product': {'available': True,
   'compare_at_price': None,
   'compare_at_price_max': 0,
   'compare_at_price_min': 0,
   'compare_at_price_varies': False,
   'content': '<span>- Acrylic lenses </span><br><span>- Lightweight and impact resistant lenses </span><br><span>- Large, oversized circular shape </span>',
   'created_at': '2016-02-10T13:53:46+11:00',
   'description': '<span>- Acrylic lenses </span><br><span>- Lightweight and impact resistant lenses </span><br><span>- Large, oversized circular shape </span>',
   'featured_image': '//cdn.shopify.com/s/files/1/0053/6742/products/10.jpg?v=1455075040',
   'handle': 'last-chance-black',
   'id': 3727646661,
   'images': ['//cdn.shopify.com/s/files/1/0053/6742/products/10.jpg?v=1455075040',
    '//cdn.shopify.com/s/files/1/0053/6742/products/9.jpg?v=1455075040'],
   'options': ['COLOUR', 'SIZE'],
   'price': 4900,
   'price_max': 49

# Use-case: non-double quoted strings

In [26]:
# adapted from https://stackoverflow.com/questions/33932654/
js = r'''
dataLayer = [{'place': 'paris', 
        'description': 'its one of the world\'s beautiful places.' }];
'''

In [27]:
import re, json
m = re.search("dataLayer = (\[.*?\]);", js, re.DOTALL | re.MULTILINE)
m.group(1)

"[{'place': 'paris', \n        'description': 'its one of the world\\'s beautiful places.' }]"

In [28]:
json.loads(m.group(1))

ValueError: Expecting property name: line 1 column 3 (char 2)

Ouch...

In [30]:
print(re.sub(r"('\B)|(\B')", '"', m.group(1)))

[{"place": "paris", 
        "description": "its one of the world\"s beautiful places." }]


In [31]:
json.loads(re.sub(r"('\B)|(\B')", '"', m.group(1)))

[{u'description': u'its one of the world"s beautiful places.',
  u'place': u'paris'}]

The js2xml way...

In [32]:
jstree = js2xml.parse(js)
print(js2xml.pretty_print(jstree))

<program>
  <assign operator="=">
    <left>
      <identifier name="dataLayer"/>
    </left>
    <right>
      <array>
        <object>
          <property name="place">
            <string>paris</string>
          </property>
          <property name="description">
            <string>its one of the world's beautiful places.</string>
          </property>
        </object>
      </array>
    </right>
  </assign>
</program>



In [34]:
values = jstree.xpath('//assign[left/identifier/@name="dataLayer"]/right/*')
values

[<Element array at 0x7f4719638290>]

In [36]:
# once you have an <array>, use make_dict()
js2xml.jsonlike.make_dict(values[0])

[{'description': "its one of the world's beautiful places.", 'place': 'paris'}]

### Another example with non-string keys

In [37]:
# from https://www.mcdonalds.com.sg/locate-us/
mcdonalds = r'''
<script>
  var _MCD = _MCD || {};
  _MCD.Config = _MCD.Config || {};
  
  _MCD.Config.StoreLocator = {
    service_root: 'https://www.mcdonalds.com.sg/wp/wp-admin/admin-ajax.php?',
    store_types: [
      { id: 1, slug: '24-hrs', label: '24-hrs', icon: 'https://d1nqx6es26drid.cloudfront.net/app/assets/img/icon_locate_us_24_hours.png' },
      { id: 2, slug: '24-hrs-weekend', label: '24-hrs Weekend', icon: 'https://d1nqx6es26drid.cloudfront.net/app/assets/img/icon_locate_us_24_hours_weekend.png' },
      { id: 3, slug: 'dessert-kiosk', label: 'Dessert Kiosk', icon: 'https://d1nqx6es26drid.cloudfront.net/app/assets/img/icon_locate_us_dessert_kiosk.png' },
      { id: 4, slug: 'drive-thru', label: 'Drive-Thru', icon: 'https://d1nqx6es26drid.cloudfront.net/app/assets/img/icon_locate_us_drive_through.png' },
      { id: 5, slug: 'live-screening', label: 'Live Screening', icon: 'https://d1nqx6es26drid.cloudfront.net/app/assets/img/icon_locate_us_live_screening.png' },
      { id: 6, slug: 'mccafe', label: 'McCafe', icon: 'https://d1nqx6es26drid.cloudfront.net/app/assets/img/icon_locate_us_mc_cafe.png' },
      { id: 7, slug: 'birthday-restaurant', label: 'Birthday Restaurant', icon: 'https://d1nqx6es26drid.cloudfront.net/app/assets/img/icon_locate_us_birthday_restaurant.png' },
      { id: 8, slug: 'create-your-taste', label: 'Create Your Taste', icon: 'https://d1nqx6es26drid.cloudfront.net/app/assets/img/icon_locate_us_create_your_taste.png' }
    ],
    modules: {
      Result: {
        flag: 'https://d1nqx6es26drid.cloudfront.net/app/assets/img/storeflag.png'
      }
    }
  };
</script>
'''

In [38]:
import scrapy 

selector = scrapy.Selector(text=mcdonalds, type="html")
jssnippet = selector.xpath('string(//script/text())').extract_first()

In [39]:
jstree = js2xml.parse(jssnippet)

In [40]:
print(js2xml.pretty_print(jstree))

<program>
  <var name="_MCD">
    <binaryoperation operation="||">
      <left>
        <identifier name="_MCD"/>
      </left>
      <right>
        <object/>
      </right>
    </binaryoperation>
  </var>
  <assign operator="=">
    <left>
      <dotaccessor>
        <object>
          <identifier name="_MCD"/>
        </object>
        <property>
          <identifier name="Config"/>
        </property>
      </dotaccessor>
    </left>
    <right>
      <binaryoperation operation="||">
        <left>
          <dotaccessor>
            <object>
              <identifier name="_MCD"/>
            </object>
            <property>
              <identifier name="Config"/>
            </property>
          </dotaccessor>
        </left>
        <right>
          <object/>
        </right>
      </binaryoperation>
    </right>
  </assign>
  <assign operator="=">
    <left>
      <dotaccessor>
        <object>
          <dotaccessor>
            <object>
              <identifier name="_M

Say you want to get all strings

In [41]:
jstree.xpath('//string/text()')

['https://www.mcdonalds.com.sg/wp/wp-admin/admin-ajax.php?',
 '24-hrs',
 '24-hrs',
 'https://d1nqx6es26drid.cloudfront.net/app/assets/img/icon_locate_us_24_hours.png',
 '24-hrs-weekend',
 '24-hrs Weekend',
 'https://d1nqx6es26drid.cloudfront.net/app/assets/img/icon_locate_us_24_hours_weekend.png',
 'dessert-kiosk',
 'Dessert Kiosk',
 'https://d1nqx6es26drid.cloudfront.net/app/assets/img/icon_locate_us_dessert_kiosk.png',
 'drive-thru',
 'Drive-Thru',
 'https://d1nqx6es26drid.cloudfront.net/app/assets/img/icon_locate_us_drive_through.png',
 'live-screening',
 'Live Screening',
 'https://d1nqx6es26drid.cloudfront.net/app/assets/img/icon_locate_us_live_screening.png',
 'mccafe',
 'McCafe',
 'https://d1nqx6es26drid.cloudfront.net/app/assets/img/icon_locate_us_mc_cafe.png',
 'birthday-restaurant',
 'Birthday Restaurant',
 'https://d1nqx6es26drid.cloudfront.net/app/assets/img/icon_locate_us_birthday_restaurant.png',
 'create-your-taste',
 'Create Your Taste',
 'https://d1nqx6es26drid.cloudfr

Say you want to grab `_MCD.Config.StoreLocator`

In [42]:
# you have to know the XML structure a bit...
jstree.xpath(r'''
    //assign[left//property/identifier/@name="StoreLocator"]
        /right/*''')

In [43]:
# use make_dict() on an <object>
# (pick first element, XPath returns node-sets)
js2xml.jsonlike.make_dict(
    jstree.xpath(r'''
        //assign[left//property/identifier/@name="StoreLocator"]
            /right/*''')[0])

{'modules': {'Result': {'flag': 'https://d1nqx6es26drid.cloudfront.net/app/assets/img/storeflag.png'}},
 'service_root': 'https://www.mcdonalds.com.sg/wp/wp-admin/admin-ajax.php?',
 'store_types': [{'icon': 'https://d1nqx6es26drid.cloudfront.net/app/assets/img/icon_locate_us_24_hours.png',
   'id': 1,
   'label': '24-hrs',
   'slug': '24-hrs'},
  {'icon': 'https://d1nqx6es26drid.cloudfront.net/app/assets/img/icon_locate_us_24_hours_weekend.png',
   'id': 2,
   'label': '24-hrs Weekend',
   'slug': '24-hrs-weekend'},
  {'icon': 'https://d1nqx6es26drid.cloudfront.net/app/assets/img/icon_locate_us_dessert_kiosk.png',
   'id': 3,
   'label': 'Dessert Kiosk',
   'slug': 'dessert-kiosk'},
  {'icon': 'https://d1nqx6es26drid.cloudfront.net/app/assets/img/icon_locate_us_drive_through.png',
   'id': 4,
   'label': 'Drive-Thru',
   'slug': 'drive-thru'},
  {'icon': 'https://d1nqx6es26drid.cloudfront.net/app/assets/img/icon_locate_us_live_screening.png',
   'id': 5,
   'label': 'Live Screening',
 

### Yet another example with var assignment

In [ ]:
# from https://www.whoscored.com/Teams/32/
# for https://stackoverflow.com/questions/25714030
js = r'''
        var defaultTeamPlayerStatsConfigParams = {
            data:{
                url: 'stage-player-stat'
            },
            defaultParams: {
                stageId: 12496,
                teamId: 32,
                playerId: -1,
                field: 2
            },
            fixZeros: true,
            fitText: {
                container: '.grid .team-link, .grid .player-link',
                options: {
                    width: 150
                }
            }
        };

'''

In [ ]:
jstree = js2xml.parse(js)

In [ ]:
jstree.xpath('//var[@name="defaultTeamPlayerStatsConfigParams"]')

In [ ]:
jstree.xpath('//var[@name="defaultTeamPlayerStatsConfigParams"]/object')

In [ ]:
obj = jstree.xpath('''
    //var[@name="defaultTeamPlayerStatsConfigParams"]
        /object''')[0]

js2xml.jsonlike.make_dict(obj)

### Live fetch example (please, demo gods...)

In [44]:
# from https://stackoverflow.com/questions/26947901
# http://www.carzone.ie/used-cars/page/1
import requests
r = requests.get('http://www.carzone.ie/used-cars/page/1')

# the page has things like
# <script id="data" type="text/javascript">
#      window.jsonData={"filters":[{"id":"keyword","groups":["keyword","e"],"texts":["eg. GTI, Sport, Sat Nav, Air bags"],"tagText":"Keywords

In [45]:
import parsel
selector = parsel.Selector(r.text, type="html")

In [46]:
for script in selector.xpath('''
        //script[@type="text/javascript"]/text()''').extract():
    jstree = js2xml.parse(script)
    d = jstree.xpath(r'''
        //assign[left/dotaccessor[object/identifier/@name="window"]
                                 [property/identifier/@name="jsonData"]]
            /right/*''')
    if d:
        data = js2xml.jsonlike.getall(d[0])
        break

In [48]:
data[0].keys()

['tealiumProperties',
 'omnitureProperties',
 'breadcrumbs',
 'filteringNeedsRedirect',
 'results',
 'heading',
 'featuredDealer',
 'bannerData',
 'state',
 'filters',
 'moduleAdverts',
 'panels',
 'googleAdsData',
 'expiredAd']

In [49]:
print(len(data[0]['results']))
data[0]['results']

10


[{'advertiserId': '8aa13d21476839cb014781aa1db446e1',
  'comparePanelUrl': 'http://www.carzone.ie/used-cars/aixam/mega-city/used-2007-aixam-mega-city-electri-cork-fpa-201614242525123?SOURCE_ID=SOURCE_ID_AD_IN_VEHICLE_COMPARE',
  'engine': '0.5 Electric',
  'features': 'CD Player, Electric Windows, Parking Sensors, Sports Suspension, Alloy Wheels',
  'financeUrl': 'http://www.carzone.ie/partners/finance/201614242525123',
  'id': '2c929b9a5438f4fb0154c8e44f85646f',
  'imageCount': 9,
  'insuranceUrl': 'http://www.carzone.ie/partners/insurance/201614242525123',
  'location': 'Cork',
  'mainImageUrl': 'http://pictures.autotrader.ie/imgser-ie/servlet/media?id=14787915&width=153&height=115&defaultImageUrl=http://www.carzone.ie/web/images/no-photo-small.gif',
  'make': 'AIXAM',
  'mileageKm': '11,600',
  'mileageMile': '7,208',
  'model': 'MEGA CITY',
  'numericId': 201614242525123,
  'pSL': True,
  'price': [{'eur': '&euro;2,995'}],
  'privateSeller': True,
  'title': 'AIXAM MEGA CITY Electr

# Using js2xml

* get it on PyPI: `pip install js2xml`
* source code on https://github.com/redapple/js2xml
* based on [`slimit`](https://github.com/rspivak/slimit) (JavaScript minifier written in Python)
* has some parsing issues sometimes
* not fast (e.g. Instagram pages)
* does not work on Python 3 (not-tested)

# Future

* replace `slimit` with a more robust and faster JavaScript parser? (e.g. [pyjsparser](https://github.com/PiotrDabkowski/pyjsparser))
* pyjsparser outputs [ESTree](https://github.com/estree/estree) trees (~ SpiderMonkey API), may obsolete js2xml altogether
* or port [acorn.js](https://github.com/ternjs/acorn) to Python (a bit like what [js-beautifier](https://github.com/beautify-web/js-beautify) did)
* [Js2Py](https://github.com/PiotrDabkowski/Js2Py) is a Pure Python JavaScript Translator/Interpreter, and looks pretty cool!

# Questions?

Note: this talk was done with a Jupyter Notebook + RISE extension